# Part H: Tracking and Homographies - Zuohe Zheng

In this part we use Practical 9c to track the positions of the four corners of the square and project a cube into the images. 

TO DO: QUESTIONS TO THINK ABOUT...

- Do the results look realistic? 
- If not then what factors do you think might be causing this


TO DO: your routines for computing a homography and extracting a valid rotation and translation go in the code below. Tips:
- you may define functions for T and H matrices respectively.
- you may need to turn the points into homogeneous form before any other computation. 
- you may need to solve a linear system in Ah = 0 form. Write your own routines or using the builtin function 'svd'. 
- you may apply the direct linear transform (DLT) algorithm to recover the best homography H.
- you may explain what & why you did in the report.


In [ ]:
import numpy as np
import scipy as sp
from scipy import io
import cv2 as cv 
import matplotlib.pyplot as plt
import pylab
import os # for reading all files in a folder
pylab.rcParams['figure.figsize'] = (12.0, 10.0)

In [ ]:
# instead of running the code here, you can also save the output of each function in a numpy array in HW2_Practical9c 
# and load it here. This could be handy if you need different hyperparameters for each corner.

# LLs = HW2_Practical9c( 'll' )
# LRs = HW2_Practical9c( 'lr' )
# ULs = HW2_Practical9c( 'ul' )
# URs = HW2_Practical9c( 'ur' )

LLs=np.load("r_ll.npy")
LRs=np.load("r_lr.npy")
ULs=np.load("r_ul.npy")
URs=np.load("r_ur.npy")

In [ ]:
#The goal of this function is to project points in XCart through projective camera
#defined by intrinsic matrix K and extrinsic matrix T.
def projectiveCamera(K,T,XCart):
    
    # TO DO: Replace this
    # XImCart =

    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    # by appending a row of 1 to the original point coordinates to form [u,v,w,1]
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    # = lambda * [x',y',1, 1/lambda]
    xCamHom1 = T @ XHom
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    # = lambda * [x',y',1]
    xCamHom = xCamHom1[0:3,:]
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    # = lambda * [x,y,1]
    xImHom = K @ xCamHom
    # TO DO: Convert points back to Cartesian coordinates xImCart
    # x = (lambda * x) / lambda
    # y = (lambda * y) / lambda
    XImCart = xImHom[0:2,:] / np.tile([xImHom[2,:]],(2,1))
    return XImCart

In [ ]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0   
    h = np.zeros(shape = [np.size(A),1])
    [U,L,Vt] = np.linalg.svd(A)
    V = np.transpose(Vt)
    h = V[:,-1]
    return h

In [ ]:
# Goal of function is to estimate pose of plane relative to camera (extrinsic matrix)
# given points in image xImCart, points in world XCart and intrinsic matrix K

def estimatePlanePose(XImCart,XCart,K):

    # TO DO: replace this
    #T = 

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    # by appending a row of 1 to the original point coordinates to form [x,y,1]
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    # multiply with inverse of intrinsic matrix
    XCamHom = np.linalg.inv(K) @ XImHom
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    
    # Extract the u and v coordinates of Cartesian 3d points XCart
    XCart = XCart[0:2,:]
    
    # Convert XCart to homogeneous coordinates XCartHom
    # [u,v] to [u,v,1]
    XCartHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    
    # Then construct the matrix A, size (n_points,9) 
    n_points = np.shape(XCamHom)[1]
    A = np.zeros(shape = [n_points*2,9])
    for n in range(n_points):
        A[2*n,[0,1,2]] = 0
        A[2*n,[3,4,5]] = -XCartHom[:,n]
        A[2*n,[6,7,8]] = XCartHom[:,n] * XCamHom[1,n]
        A[2*n+1,[0,1,2]] = XCartHom[:,n]
        A[2*n+1,[3,4,5]] = 0
        A[2*n+1,[6,7,8]] = -XCartHom[:,n] * XCamHom[0,n]
    
    # Solve Ah = 0
    h = solveAXEqualsZero(A)
    # Reshape h into the matrix H, values of h go first into rows of H
    h_width = np.sqrt(np.size(h))
    h_width = int(h_width)
    H = np.reshape(h,[h_width,h_width])
     
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD
    # SVD decomposition of the first two columns of H
    [U,L,Vt] = np.linalg.svd(H[:,0:2])
    R = np.zeros(shape=[h_width,h_width])
    # Replace L with [1,0;0,1;0,0] to form the first two columns of R
    l = np.array([[1,0],[0,1],[0,0]])
    R[:,0:2] = U @ l @ Vt

    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R[:,2] = np.cross(R[:,0],R[:,1])
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    if np.linalg.det(R) <= 0 :
        R[:,-1] = - R[:,-1]
    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    # Find translation scaling factor between old and new values
    Lamb = H / R
    lamb = np.sum(Lamb[0:3,0:2]) / 6
    t = H[:,-1] / lamb
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[-1] < 0 :
        t = -t
        R[:,0:2] = -R[:,0:2]
    
            
    # TO DO: Assemble transformation into matrix form
    # append [Tx,Ty,Tz] to R as the last column
    # then append [0,0,0,1] as the last row to form the final transformation matrix
    t = np.reshape(t,[3,1])
    T = np.concatenate((R, t), axis=1)
    T = np.concatenate((T, np.array([[0,0,0,1]])), axis=0)
    
    return T 

In [ ]:
# Load all images in folder
images = []
nFrame = 0
folder = 'Pattern01/'
for frameNum in sorted(os.listdir(folder)):
    images.append(cv.imread(folder+frameNum))
    nFrame += 1
# # plot first image 
# plt.imshow(images[0])
# plt.show()


# Coordinates of the known target object (a dark square on a plane) in 3D:
XCart = np.array([[-50, -50,  50,  50],
          [50, -50, -50,  50],
            [0, 0, 0, 0]])

# These are some approximate intrinsics for this footage.
K = np.array([[640, 0, 320],
          [0, 512, 256],
            [0, 0, 1]])

# Define 3D points of wireframe object.
XWireFrameCart = np.array([[-50, -50,  50,  50, -50, -50,  50,  50],
          [50, -50, -50,  50, 50, -50, -50,  50],
            [0, 0, 0, 0, -100, -100, -100, -100, ]])

In [ ]:
# ================================================
for iFrame in range(nFrame):
    print('Processing Frame', iFrame)
    xImCart = np.array([LLs[iFrame,:].T, ULs[iFrame,:].T, URs[iFrame,:].T, LRs[iFrame,:].T]).T

    # get a frame from footage 
    im = images[iFrame]

    # Draw image and 2d points
    plt.imshow(im)
    plt.scatter(x = xImCart[0,:], y = xImCart[1,:],c = 'r')
#     plt.show()


    #TO DO: Use your routine to calculate TEst the extrinsic matrix relating the
    #plane position to the camera position.
    #T = estimatePlanePose(xImCart, XCart, K);
    TEst = estimatePlanePose(xImCart,XCart,K)
    
    # TO DO: Draw a wire frame cube using data XWireFrameCart. You need to
    # 1) project the vertices of a 3D cube through the projective camera;
    # 2) draw lines betweeen the resulting 2d image points.
    # Note: CONDUCT YOUR CODE FOR DRAWING XWireFrameCart HERE
    
    XWireFrameCartProjected = projectiveCamera(K,TEst,XWireFrameCart)
    plt.plot(XWireFrameCartProjected[0,],XWireFrameCartProjected[1,],'g.')

    # Connect the points to draw the frame of the cube
    for cPoint in range(4):
        plt.plot([XWireFrameCartProjected[0,cPoint],XWireFrameCartProjected[0,cPoint+4]], \
                 [XWireFrameCartProjected[1,cPoint],XWireFrameCartProjected[1,cPoint+4]],'g-') 
    for cPoint in range(3):
        plt.plot([XWireFrameCartProjected[0,cPoint],XWireFrameCartProjected[0,cPoint+1]], \
                 [XWireFrameCartProjected[1,cPoint],XWireFrameCartProjected[1,cPoint+1]],'g-') 
        plt.plot([XWireFrameCartProjected[0,cPoint+4],XWireFrameCartProjected[0,cPoint+5]], \
                 [XWireFrameCartProjected[1,cPoint+4],XWireFrameCartProjected[1,cPoint+5]],'g-')
    plt.plot([XWireFrameCartProjected[0,0],XWireFrameCartProjected[0,3]], \
                 [XWireFrameCartProjected[1,0],XWireFrameCartProjected[1,3]],'g-') 
    plt.plot([XWireFrameCartProjected[0,4],XWireFrameCartProjected[0,7]], \
                 [XWireFrameCartProjected[1,4],XWireFrameCartProjected[1,7]],'g-') 

    plt.axis('off')
    plt.show()


The figures above show the result of four corners of the black square tracked using condensation algorithm and the wireframe cube biult based on the tracking points and homography estimation. The tracking results in all frames are correct and most of cubes built are realistic except for the homography transformation works bad in a few frames because of the characteristic of equation system.

In this case, distribution region can be restricted to simplify and improve the tracking. For example, when tracking upper right corner, likelihood or particles can be simply restricted to only the upper right region of the current frame. An elegant way is to use edge detector, which is implemented in another file.

A basic method to improve the tracking result is adjust particle number. A number around 3000 of particles can have a good tracking result. The lower number of particles may not cover the all image and is easy to cause error or missings, but using too many particles is redundant. Different numbers of particles were chosen for four corners. 

Another way is to adjust the noise level with the moving velocity of camera. When the camera move fast, the objects in the scene have large displacements and the particles with small distribution range are hard to keep pace with the moving object. So lower noise std can be used when the image is still, to make particles concentrated, and higher noise std when camera moves fast. Calculating displacement before adding noise to new particles can be helpful.

Compared to the street scene used in task9B, the likelihood figures of matching black corners are obviously more clear and concentrated. The MAP estimation performance better when the template and the image are simple. Using other template match algorithms like non-local-means can also improve the tracking results.